In [1]:
import json
inputs = json.load(open('inputs.json'))

In [2]:
# make sure we're in project directory where the data is sitting
import os
os.chdir(os.environ['PROJECT_DIR'])

In [3]:
# load scores
import pandas as pd
from cStringIO import StringIO
import zipfile
with zipfile.ZipFile('ca2015_1_csv_v3.zip') as zf:
    f = StringIO(zf.read('ca2015_1_csv_v3.txt'))
    make_int = lambda x: int(x) if x != '*' else 0
    df = pd.read_csv(f,
                usecols=['Grade', 'Test Id', 'County Code', 'District Code', 'School Code', 'Percentage Standard Exceeded', 'Percentage Standard Met', 'Percentage Standard Nearly Met', 'Percentage Standard Not Met'],
                dtype={'County Code' : str, 'District Code': str, 'School Code' : str }, 
                converters={'Percentage Standard Exceeded' : make_int, 'Percentage Standard Met' : make_int,
                       'Percentage Standard Nearly Met' : make_int, 'Percentage Standard Not Met' : make_int,
                      })

In [4]:
# merge school information with scores
school_df = pd.read_csv('pubschls.zip', sep='\t', header=0, dtype={'CDSCode' : str}, 
                                usecols='CDSCode District Latitude Longitude School'.split())

school_df.set_index('CDSCode', inplace=True)

df['CDSCode'] = df['County Code'] + df['District Code'] + df['School Code']
df.set_index('CDSCode', inplace=True)

joined_df = df.join(school_df, how='inner')

In [5]:
df.columns

Index([u'County Code', u'District Code', u'School Code', u'Grade', u'Test Id',
       u'Percentage Standard Exceeded', u'Percentage Standard Met',
       u'Percentage Standard Nearly Met', u'Percentage Standard Not Met'],
      dtype='object')

In [6]:
df.Grade.unique()

array([ 3,  4,  5,  6,  7,  8, 11, 13])

In [7]:
df['Test Id'].unique()

array([2, 1])

In [8]:
# use the Google Maps API to a GPS coordinate from an address
def get_geocoordinate(search_text):
    import urllib, urllib2
    search_text = search_text.strip()
        
    geocode = json.loads(urllib2.urlopen(
        'http://maps.googleapis.com/maps/api/geocode/json?address=%s&sensor=false' % urllib.quote_plus(search_text)).read()
                     )
    loc = geocode['results'][0]['geometry']['location'] # {u'lat': 37.000, u'lng': -122.000}
        
    return loc

In [9]:
loc = get_geocoordinate(inputs['address'])

In [10]:
# subset data based on location, grade, and subject
def view_scores_in_neighborhood(loc, grade, subject, limit=20):
    from scipy.spatial.distance import cdist
    
    test_id = 1 if subject == 'English' else 2

    joined_df['dist'] = cdist([[loc['lat'], loc['lng']]], joined_df[['Latitude', 'Longitude']])[0]

    matches = (joined_df.Grade == grade) & (joined_df['Test Id'] == test_id) & (joined_df['School'].notnull())
    subset = joined_df[matches].sort_values(by='dist').head(limit).sort_values(by='District Code')
    
    return subset

In [11]:
subset = view_scores_in_neighborhood(loc, inputs['grade'], inputs['subject'])

In [12]:
import cufflinks as cf
#cf.go_offline(connected=True)
# pypi version of cufflinks doesn't include the connected option, so paste relevant code:
# https://github.com/santosjorge/cufflinks/commit/730f777cfeca233b63d774361bb96616f901b33e
import plotly.offline as py_offline
py_offline.init_notebook_mode(connected=True)
py_offline.__PLOTLY_OFFLINE_INITIALIZED=True

In [13]:
# get data columns
cols = subset.columns[subset.columns.str.contains('Percentage')]

# plot
subset.set_index('School', inplace=True) # show school on x axis
subset[cols].iplot(kind='bar', barmode='stack', 
     layout_update=dict(xaxis=dict(tickangle=45), margin=dict(b=120)))

In [14]:
import folium
fmap = folium.Map(location=[loc['lat'], loc['lng']], tiles='Stamen Toner', zoom_start=12, max_zoom=15)
for i, (index, row) in enumerate(subset.iterrows()):
    fmap.add_children(
        folium.CircleMarker([row['Latitude'], row['Longitude']], radius=200, 
                       fill_color='blue', popup=index))
fmap